In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
import os
os.chdir("../raw data")

In [3]:
# importing fips data. and rearranging the columns
fips = pd.read_excel("fips.xlsx")
fips = fips.iloc[:, [1, 3, 2, 4, 5]]

# Convert the 'integers' to integers in fips file.
for i in range(fips.shape[0]):
    if fips.iloc[i, 4] != 'unknown':
        fips.iloc[i, 4] = int(fips.iloc[i, 4])
        fips.iloc[i, 2] = int(fips.iloc[i, 2])



In [21]:
# importing the nyt data. and rearrangign the columns
nyt = pd.read_excel("nyt.xlsx")
nyt = nyt.iloc[:, [0, 3, 4, 5, 2, 1]]
nyt.head()

In [30]:
fips.head()

,stfips,stname,ctyfips,ctyname,fips
0,1,Alabama,0,0,0
1,1,Alabama,1,Autauga County,1001
2,1,Alabama,3,Baldwin County,1003
3,1,Alabama,5,Barbour County,1005
4,1,Alabama,7,Bibb County,1007


In [31]:
# making the columns for each row. date_cases and date_deaths

list_of_dates = []
for i in range(nyt.shape[0]):
    if (nyt.iloc[i, 0] in list_of_dates) == False:
        list_of_dates.append(nyt.iloc[i, 0])

date_case = [item + '_cases' for item in list_of_dates]
date_death = [item + '_deaths' for item in list_of_dates]

# Assigning values to each column to be 0 initially

for item in date_case:
    fips[item] = 0
for item in date_death:
    fips[item] = 0

In [42]:
fips.head()

,stfips,stname,ctyfips,ctyname,fips,2020-01-21_cases,2020-01-22_cases,2020-01-23_cases,2020-01-24_cases,2020-01-25_cases,...,2020-03-28_deaths,2020-03-29_deaths,2020-03-30_deaths,2020-03-31_deaths,2020-04-01_deaths,2020-04-02_deaths,2020-04-03_deaths,2020-04-04_deaths,2020-04-05_deaths,2020-04-06_deaths
0,1,Alabama,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Alabama,1,Autauga County,1001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,Alabama,3,Baldwin County,1003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,Alabama,5,Barbour County,1005,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,Alabama,7,Bibb County,1007,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
# function to get row number of a county in fips. Only for Known counties
# Input is the nyt row number. This function will convert i to the county number. You get the row number for that county in the fips df

def row_loc_county(i):
    inp = nyt.iloc[i, 1]
    location = fips.index[(fips['fips']) == inp].values
    location = location.tolist()
    return location

def row_loc_unknown(i):
    inp = nyt.iloc[i, 4]
    sliced = fips.loc[fips['stname'] == inp]
    location = sliced.index[sliced['fips'] == 'unknown'].values
    location = location.tolist()
    return location

# function to get column of a date_case for a given row number in nyt data
# Input is nyt row number. This function will take the date from that, add "cases" and find the column in fips df

def col_loc_case(i):
    location = fips.columns.get_loc(nyt.iloc[i, 0] + "_cases")
    return location

# function to get column of a date_death for a given row number in nyt data
# input is a nyt row number. This function will take the date from that, add "deaths" and find the column in fips df

def col_loc_death(i):
    location = fips.columns.get_loc(nyt.iloc[i, 0] + "_deaths")
    return location

In [88]:
nyt.head()

,date,fips,cases,deaths,state,county
0,2020-01-21,53061.0,1,0,Washington,Snohomish
1,2020-01-22,53061.0,1,0,Washington,Snohomish
2,2020-01-23,53061.0,1,0,Washington,Snohomish
3,2020-01-24,17031.0,1,0,Illinois,Cook
4,2020-01-24,53061.0,1,0,Washington,Snohomish


In [91]:
for i in range(nyt.shape[0]):
    if nyt.iloc[i, 5] != 'Unknown':
        fips.iloc[row_loc_county(i), col_loc_case(i)] = nyt.iloc[i, 2]
        fips.iloc[row_loc_county(i), col_loc_death(i)] = nyt.iloc[i, 3]
    else:
        fips.iloc[row_loc_unknown(i), col_loc_case(i)] = nyt.iloc[i, 2]
        fips.iloc[row_loc_unknown(i), col_loc_death(i)] = nyt.iloc[i, 3]


In [4]:
os.chdir("../data")

In [5]:
fips.to_csv("newNYT.csv")